In [35]:
#!/usr/bin/python
import pymysql
import pandas as pd
import numpy as np
import datetime as dt
import io
from impala.dbapi import connect as impala_conn
from impala.util import as_pandas

def try_decode(cell):
    try:
        return cell.decode(encoding = "utf-8")
    except AttributeError:
        return cell
    
def decode_data(d):
    return [{try_decode(key): try_decode(val) for key, val in item.items()} for item in d]

def query_db(query, db = "mariadb", fmt = "pandas"):
    if db not in ["mariadb", "hadoop"]:
        raise ValueError("The db should be `mariadb` or `hadoop`.")
    if fmt not in ["pandas", "raw"]:
        raise ValueError("The format should be either `pandas` or `raw`.")
    
    if db == "mariadb":
        try:
            conn = pymysql.connect(
                host = "analytics-store.eqiad.wmnet",
                read_default_file = '/etc/mysql/conf.d/research-client.cnf',
                charset = 'utf8mb4',
                db='staging',
                cursorclass=pymysql.cursors.DictCursor
            )
            if fmt == "pandas":
                result = pd.read_sql_query(query, conn)
                # Turn any binary data into strings
                result = result.applymap(try_decode)
            elif fmt == "raw":
                cursor = conn.cursor()
                cursor.execute(query)
                result = cursor.fetchall()
                result = decode_data(result)
        finally:
            conn.close()
        
    elif db == "hadoop":
        try:
            hive_conn = impala_conn(host='analytics1003.eqiad.wmnet', port=10000, auth_mechanism='PLAIN')
            hive_cursor = hive_conn.cursor()
            hive_cursor.execute(query)
            if fmt == "pandas":
                try:
                    result = as_pandas(hive_cursor)
                # Happens if there are no results (as with an INSERT INTO query)
                except TypeError:
                    pass
            elif fmt == "raw":
                result = hive_cursor.fetchall()
        finally:
            hive_conn.close()
    
    try:
        return result
    except UnboundLocalError:
        pass

In [30]:
articles_q = """
select database() as wiki, ss_good_articles as articles
from site_stats;
"""

with open("num_articles.sql", mode = "w") as f:
    f.write(articles_q)

# Wikipedia articles

In [31]:
wp_dbs = query_db("select site_global_key from enwiki.sites where site_group = 'wikipedia'")
wp_dbs.to_csv("wikipedia_dbs.tsv", sep = "\t", header = False, index = False)

In [32]:
%%bash --out articles_by_wp
multiquery num_articles.sql -h analytics-store.eqiad.wmnet -d wikipedia_dbs.tsv \
--defaults-file=/etc/mysql/conf.d/research-client.cnf

Executing query on aawiki
Executing query on abwiki
Executing query on acewiki
Executing query on afwiki
Executing query on akwiki
Executing query on alswiki
Executing query on amwiki
Executing query on anwiki
Executing query on angwiki
Executing query on arwiki
Executing query on arcwiki
Executing query on arzwiki
Executing query on aswiki
Executing query on astwiki
Executing query on avwiki
Executing query on aywiki
Executing query on azwiki
Executing query on bawiki
Executing query on barwiki
Executing query on bat_smgwiki
Executing query on bclwiki
Executing query on bewiki
Executing query on be_x_oldwiki
Executing query on bgwiki
Executing query on bhwiki
Executing query on biwiki
Executing query on bjnwiki
Executing query on bmwiki
Executing query on bnwiki
Executing query on bowiki
Executing query on bpywiki
Executing query on brwiki
Executing query on bswiki
Executing query on bugwiki
Executing query on bxrwiki
Executing query on cawiki
Executing query on cbk_zamwiki
Executing 

In [36]:
art = pd.read_table(io.StringIO(initial_value = articles_by_wp))

In [38]:
art["articles"].sum()

46736979

# Wiktionary entries

In [39]:
wp_dbs = query_db("select site_global_key from enwiki.sites where site_group = 'wiktionary'")
wp_dbs.to_csv("wiktionary_dbs.tsv", sep = "\t", header = False, index = False)

In [40]:
%%bash --out entries_by_wt
multiquery num_articles.sql -h analytics-store.eqiad.wmnet -d wiktionary_dbs.tsv \
--defaults-file=/etc/mysql/conf.d/research-client.cnf

Executing query on aawiktionary
Executing query on abwiktionary
Executing query on afwiktionary
Executing query on akwiktionary
Executing query on alswiktionary
Executing query on amwiktionary
Executing query on anwiktionary
Executing query on angwiktionary
Executing query on arwiktionary
Executing query on aswiktionary
Executing query on astwiktionary
Executing query on avwiktionary
Executing query on aywiktionary
Executing query on azwiktionary
Executing query on bewiktionary
Executing query on bgwiktionary
Executing query on bhwiktionary
Executing query on biwiktionary
Executing query on bmwiktionary
Executing query on bnwiktionary
Executing query on bowiktionary
Executing query on brwiktionary
Executing query on bswiktionary
Executing query on cawiktionary
Executing query on chwiktionary
Executing query on chrwiktionary
Executing query on cowiktionary
Executing query on crwiktionary
Executing query on cswiktionary
Executing query on csbwiktionary
Executing query on cywiktionary
Exe

In [41]:
ent = pd.read_table(io.StringIO(initial_value = entries_by_wt))

In [43]:
ent["articles"].sum()

27409180

# Wikidata items

In [45]:
query_db("select ss_good_articles from wikidatawiki.site_stats")

,ss_good_articles
0,38687601


# Commons files

In [8]:
licenses_q = """
select license, count(*) as files
from
(select
  case
    when cats like '%CC-BY-NC-ND%' then 'CC-BY-NC-ND'
    when cats like '%CC-BY-NC-SA%' then 'CC-BY-NC-SA'
    when cats like '%CC-BY-ND%' then 'CC-BY-ND'
    when cats like '%CC-BY-SA%' then 'CC-BY-SA'
    when cats like '%CC-BY-NC%' then 'CC-BY-NC'
    when cats like '%CC-BY%' then 'CC-BY'
    when cats like '%CC-SA%' then 'CC-SA'
    when cats like '%CC-Zero%' then 'CC-0'
    when cats like '%CC-PD%' then 'CC-PD'
    else 'Other CC'
  end as license
  from
  (select cl_from, group_concat(cl_to) as cats
    from commonswiki.categorylinks
    inner join
      (select cat_title from commonswiki.category where
        cat_title like 'CC-%' and
        cat_title not like '%aircraft%' and
        cat_title not regexp 'CC-[[:upper:][:digit:]]{3}'
      ) cc_cats
    on cat_title = cl_to
    where cl_type = "file"
    group by cl_from
  ) cc_files
) licenses
group by license;
"""

licenses = query_db(licenses_q)

In [17]:
licenses.sort_values("license")

,license,files
0,CC-0,2010188
1,CC-BY,6104697
2,CC-BY-NC,6171
3,CC-BY-NC-SA,231
4,CC-BY-ND,15
5,CC-BY-SA,25099307
6,CC-PD,3632883
7,CC-SA,2804
8,Other CC,1


In [12]:
licenses["files"].sum()

36856297

In [ ]:
media_types_q = """
select img_media_type, count(*) as files
from
(select distinct cl_from
  from commonswiki.categorylinks
  inner join
    (select cat_title from commonswiki.category where
      cat_title like 'CC-%' and
      cat_title not like '%aircraft%' and
      cat_title not regexp 'CC-[[:upper:][:digit:]]{3}'
    ) cc_cats
  on cat_title = cl_to
  where cl_type = "file"
) cc_files
inner join commonswiki.page on cl_from = page_id
inner join commonswiki.image on page_title = img_name
group by img_media_type;
"""

media_types = query_db(media_types_q)

In [27]:
media_types

,img_media_type,files
0,BITMAP,34988944
1,DRAWING,786768
2,AUDIO,774706
3,VIDEO,97442
4,MULTIMEDIA,4
5,OFFICE,218686


In [28]:
media_types["files"].sum()

36866550

## Total Commons files

In [47]:
query_db("select ss_images from commonswiki.site_stats")

,ss_images
0,42744326
